In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib

# Load the training data
train_data = pd.read_csv("grid_stability_5years_training_data.csv")

# Convert DateTime column to datetime dtype
train_data['DateTime'] = pd.to_datetime(train_data['DateTime'])

# Drop DateTime column as it's not a feature
train_data.drop("DateTime", axis=1, inplace=True)

# Convert categorical target to numeric
train_data['stability'] = train_data['stability'].map({'stable': 1, 'unstable': 0})

# Separate features and target variable
X_train = train_data.drop("stability", axis=1)
y_train = train_data["stability"]

# Load the testing data
test_data = pd.read_csv("grid_stability_3months_validation_data.csv")

# Convert DateTime column to datetime dtype
test_data['DateTime'] = pd.to_datetime(test_data['DateTime'])

# Drop DateTime column as it's not a feature
test_data.drop("DateTime", axis=1, inplace=True)

# Ensure feature columns in testing data match those in training data
X_test = test_data[X_train.columns]

# Convert categorical target to numeric (if needed)
test_data['stability'] = test_data['stability'].map({'stable': 1, 'unstable': 0})

# Initialize and train XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

# Make predictions with the XGBoost model
predicted_stability = xgb_model.predict(X_test)

# Load the actual stability values for the testing period
actual_stability = test_data["stability"]

# Calculate Mean Squared Error and Mean Absolute Error
mse = mean_squared_error(actual_stability, predicted_stability)
mae = mean_absolute_error(actual_stability, predicted_stability)

# Calculate the range of actual stability values
range_actual_stability = max(actual_stability) - min(actual_stability)

# Compute accuracy percentage
accuracy_percentage = 100 * (1 - (mae / range_actual_stability))

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("Accuracy Percentage:", accuracy_percentage)

# Save the trained model to a .joblib file
joblib.dump(xgb_model, 'stability.joblib')
print("Model saved as 'stability.joblib'")


KeyError: 'DateTime'